In [10]:
model = "gpt-5-mini-2025-08-07"
import tiktoken
def count_tokens(text: str, model: str = model) -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

特に要望はありませんが、employee_3さんとemployee_4とは息が合っているなという印象です。
employee_5さんとだけ働きたい

In [11]:
employee_list = ["Employee_1", "Employee_2", "Employee_3", "Employee_4", "Employee_5"]
employee_flat = ", ".join([f"e{i+1}:{name}" for i, name in enumerate(employee_list)])

text_input = """
employee_2さんとは働きやすいやすいので、時々一緒に働けたら嬉しいです。
"""
main_context_1 = f"""
The available employees are: {employee_flat}. Total employees are {len(employee_list)}.
Task:
- Comprehend the given text input in Japanese and classify willingness to work with a given employee.

Output Format:
- JSON object with keys "e1" to "eN" (N = {len(employee_list)}).
- The value indicates willingness to work with that employee.
- Employees not mentioned in the text MUST default to neutral (0)."""
main_context_2 = """
- Example:
  {
  "employees": {
    "e1": float,
    "e2": float,
    "e3": float,
    ...
    "eN":float
  }
  }


Willingness Scale:
- Very unwilling: -1.0 to -0.6 (exclusive)
- Slightly unwilling: -0.6 to -0.2 (exclusive)
- Neutral: -0.2 to 0.2 (exclusive)
- Slightly willing: 0.2 to 0.6 (exclusive)
- Very willing: 0.6 to 1.0 (inclusive)

Japanese Phrase Mapping (with precedence rules):
- If a phrase contains both positive and negative (e.g., "入れるけどできれば入りたくない"), **the negative overrides the positive**.
- "入れる" or "働ける" → slightly willing (0.2–0.6)
- "できれば入りたくない" → slightly unwilling (-0.6 to -0.2)
- "無理" or "絶対無理" → very unwilling (-1.0 to -0.6)
- "一番嬉しい" or "大好き" → very willing (0.6–1.0)
Instructions:
1. Never hallucinate information; only use what is explicitly stated.
2. Identify willingness at the phrase level in the input text.
3. Resolve conflicts using the precedence rules above.
4. Only return the JSON object; do not include any explanations or extra text.
"""
main_context = main_context_1 + main_context_2
print(count_tokens(main_context+text_input))

465


In [12]:
print(main_context+text_input)


The available employees are: e1:Employee_1, e2:Employee_2, e3:Employee_3, e4:Employee_4, e5:Employee_5. Total employees are 5.
Task:
- Comprehend the given text input in Japanese and classify willingness to work with a given employee.

Output Format:
- JSON object with keys "e1" to "eN" (N = 5).
- The value indicates willingness to work with that employee.
- Employees not mentioned in the text MUST default to neutral (0).
- Example:
  {
  "employees": {
    "e1": float,
    "e2": float,
    "e3": float,
    ...
    "eN":float
  }
  }


Willingness Scale:
- Very unwilling: -1.0 to -0.6 (exclusive)
- Slightly unwilling: -0.6 to -0.2 (exclusive)
- Neutral: -0.2 to 0.2 (exclusive)
- Slightly willing: 0.2 to 0.6 (exclusive)
- Very willing: 0.6 to 1.0 (inclusive)

Japanese Phrase Mapping (with precedence rules):
- If a phrase contains both positive and negative (e.g., "入れるけどできれば入りたくない"), **the negative overrides the positive**.
- "入れる" or "働ける" → slightly willing (0.2–0.6)
- "できれば入りたくない" → 

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List, Tuple, Dict
client = OpenAI(api_key="")
from typing import List

class Employee(BaseModel):
    id: str
    willingness: float

class EmployeeWillingness(BaseModel):
    employees: List[Employee]

response = client.beta.chat.completions.parse(
    model=model,
    messages=[
        {"role": "system", "content": main_context},
        {"role": "user", "content": text_input},
    ],
    response_format=EmployeeWillingness,
)

employee_pref = response.choices[0].message.parsed

# Convert to dict if needed
employee_dict = {emp.id: emp.willingness for emp in employee_pref.employees}

In [14]:
employee_dict

{'e1': 0.0, 'e2': 0.4, 'e3': 0.0, 'e4': 0.0, 'e5': 0.0}